In [1]:
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter("ignore")

In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Using device {device}")

Using device cuda


Example on how functions from `extraction_refactored.py` can be used, especially one which calculates perplexity

In [3]:
from extraction_refactored import calculate_perplexity, parse_model_name
model_name = "bigcode/starcoderbase-1b"
model, tokenizer = parse_model_name(model_name)
model.to(device)
text1 = "import pandas as pd\nimport numpy as np"
text2 = "#include <iostream>"
perplexity_score1 = calculate_perplexity(text1, model, tokenizer).item()
perplexity_score2 = calculate_perplexity(text2, model, tokenizer).item()
print(perplexity_score1)
print(perplexity_score2)

Using device cuda
4.501964092254639
7.048807621002197


## Now exploring results we got from scripts
We download results from file with samples, then iterate through dataset chunks and check if this code is some substring.

In [7]:
from utilities import read_results, search_in_train
texts = read_results('results_full')[2]
uniques = pd.Series(texts).astype(str).drop_duplicates()
total = search_in_train('t5_datase', texts, 'code')
train_texts = [text for text in uniques if total[total['code'].apply(lambda x: text in str(x))].shape[0] > 0]

Drop duplicates of same paths and some code to see the overall picture.

In [18]:
total_no_dupl = total.drop_duplicates(['path']).drop_duplicates(['code'])
total_no_dupl.head(10)

,code,repo_name,path,language,license,size
29070,"#include ""qrcodedialog.h""\n#include ""ui_qrcode...",elco-coin/elcoin-source,src/qt/qrcodedialog.cpp,C++,mit,4309.0
225696,"/*\n * Copyright 2014-2019 Amazon.com, Inc. or...",jentfoo/aws-sdk-java,aws-java-sdk-glue/src/main/java/com/amazonaws/...,Java,apache-2.0,7408.0
272585,"/*\n * Copyright 2014-2019 Amazon.com, Inc. or...",jentfoo/aws-sdk-java,aws-java-sdk-glue/src/main/java/com/amazonaws/...,Java,apache-2.0,3381.0
580673,// \n// Copyright (c) Microsoft and contributo...,namratab/azure-sdk-for-net,src/ResourceManagement/Network/NetworkManageme...,C#,apache-2.0,3544.0
1103503,"/*\n * Copyright 2014-2019 Amazon.com, Inc. or...",jentfoo/aws-sdk-java,aws-java-sdk-glue/src/main/java/com/amazonaws/...,Java,apache-2.0,4023.0
1363794,"/*\n * Copyright 2014-2019 Amazon.com, Inc. or...",jentfoo/aws-sdk-java,aws-java-sdk-glue/src/main/java/com/amazonaws/...,Java,apache-2.0,3287.0
1659519,// \n// Copyright (c) Microsoft and contributo...,nemanja88/azure-sdk-for-net,src/ResourceManagement/Network/NetworkManageme...,C#,apache-2.0,4173.0
51777,"/*\n * Copyright 2014-2019 Amazon.com, Inc. or...",jentfoo/aws-sdk-java,aws-java-sdk-workdocs/src/main/java/com/amazon...,Java,apache-2.0,3246.0
433462,"/*\n * Copyright 2014-2019 Amazon.com, Inc. or...",jentfoo/aws-sdk-java,aws-java-sdk-glue/src/main/java/com/amazonaws/...,Java,apache-2.0,3923.0
526404,"/*\n * Copyright 2014-2019 Amazon.com, Inc. or...",jentfoo/aws-sdk-java,aws-java-sdk-glue/src/main/java/com/amazonaws/...,Java,apache-2.0,3360.0


Check for appearances of memorized texts in train set. Also consider text which is most probably was fully memorized and in train set.

In [19]:
train_counts = np.array([total[total['code'].apply(lambda x: text in str(x))].shape[0] for text in train_texts])
print(train_counts)
most_probably_train_text = train_texts[np.argmin(train_counts)]
print(most_probably_train_text)

[  6   8  37   1 233]
/*
 * Copyright 2014-2019 Amazon.com, Inc. or its affiliates. All Rights Reserved.
 * 
 * Licensed under the Apache License, Version 2.0 (the "License"). You may not use this file except in compliance with
 * the License. A copy of the License is located at
 * 
 * http://aws.amazon.com/apache2.0
 * 
 * or in the "license" file accompanying this file. This file is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR
 * CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions
 * and limitations under the License.
 */
package com.amazonaws.services.elasticfilesystem.model.transform;

import java.math.*;

import javax.annotation.Generated;

import com.amazonaws.services.elasticfilesystem.model.*;
import com.amazonaws.transform.SimpleTypeJsonUnmarshallers.*;
import com.amazonaws.transform.*;

import com.fasterxml.jackson.core.JsonToken;


In [21]:
scores_S = np.load('scores_S', allow_pickle=True)
scores_XL = np.load('scores_XL', allow_pickle=True)
scores_zlib = np.load('scores_zlib', allow_pickle=True)
scores_Lower = np.load('scores_Lower', allow_pickle=True)

In [24]:
scores_S.shape, scores_XL.shape, scores_Lower.shape, scores_zlib.shape

((40000,), (32,), (32,), (32,))